# GROUPING

After preparing the data for a more exhaustive analysis, the last step is to separate the different countries into group with common characteristics. This is necessary as an aggregate study of all countries would mean erasing the differences between them and missing important conclusions, making our study too shallow.

Grouping them by similar characteristics will simplify our analysis, all while reducing the data loss and stressing common patterns among countries in the same group.

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
import datetime
from Project.Utils.aggregate import aggregate

import warnings
warnings.filterwarnings("ignore")

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

col_index = [col_country, col_region, col_year]

read_path = os.getcwd()
region_url = read_path + '/Databases/AuxiliarData/world-regions-mod.csv'
df_url = read_path + '/Output/SilverDataframe.csv'

output_path = os.getcwd() + '/Output/'
write_path = os.getcwd() + '/Output/'
country_folder = write_path + '/Country/'
region_folder = write_path + '/Region/'
aggregate_folder = region_folder + '/Aggregate/'



## CREATING DIRECTORIES
If the destination folders do not exist, create them.

In [2]:
if not os.path.exists(country_folder):
            os.makedirs(country_folder)
if not os.path.exists(region_folder):
            os.makedirs(region_folder)
if not os.path.exists(aggregate_folder):
            os.makedirs(aggregate_folder)

## MODIFY THE REGION FILE
We downloaded a .csv file with the regions of the world according to the World Bank. However, we need to modify it slightly before using it.

Since we plan to repeatedly later, we will save this modified version.

In [3]:
if not os.path.exists(region_url) or True:

    reg_df = pd.read_csv(read_path + '/Databases/AuxiliarData/' + 'world-regions.csv')

    display(reg_df)
    reg_df.drop(columns=['Code', 'Year'], inplace = True)
    reg_df.rename(columns = {'World Region according to the World Bank': col_region, 'Entity': col_country}, inplace = True)
    reg_df.set_index([col_country, col_region], inplace = True)
    display(reg_df)

    reg_df.to_csv(region_url)


,Entity,Code,Year,World Region according to the World Bank
0,Afghanistan,AFG,2017,South Asia
1,Albania,ALB,2017,Europe and Central Asia
2,Algeria,DZA,2017,Middle East and North Africa
3,American Samoa,ASM,2017,East Asia and Pacific
4,Andorra,AND,2017,Europe and Central Asia
...,...,...,...,...
212,Venezuela,VEN,2017,Latin America and Caribbean
213,Vietnam,VNM,2017,East Asia and Pacific
214,Yemen,YEM,2017,Middle East and North Africa
215,Zambia,ZMB,2017,Sub-Saharan Africa


,
Country,Region
Afghanistan,South Asia
Albania,Europe and Central Asia
Algeria,Middle East and North Africa
American Samoa,East Asia and Pacific
Andorra,Europe and Central Asia
...,...
Venezuela,Latin America and Caribbean
Vietnam,East Asia and Pacific
Yemen,Middle East and North Africa


### READ THE FULL DATAFRAME AND THE REGION DATAFRAME

In [4]:
df = pd.read_csv(df_url, index_col = [col_country, col_year])
region_df = pd.read_csv(region_url, index_col = [col_country, col_region])

## ADD THE PROPERTY 'REGION'
By merging both DataFrames, we will obtain a DataFrame similar to the original one with a new field for the index: `region`

In [5]:
display(df)
df = pd.merge(region_df, df, how = 'inner', left_index = True, right_index = True)
display(df)

AgriShareGDP  CreditToAgriFishForest  EmploymentRural  \
Country     Year                                                          
Afghanistan 2000      54.06300                     NaN              NaN   
            2001      54.06300                     NaN              NaN   
            2002      45.13440                     NaN              NaN   
            2003      41.90340                     NaN              NaN   
            2004      35.61280                     NaN              NaN   
...                        ...                     ...              ...   
Zimbabwe    2016       7.87399                     NaN              NaN   
            2017       8.34095                     NaN              NaN   
            2018       8.30469                     NaN              NaN   
            2019       8.17322                     NaN              NaN   
            2020      10.93630                     NaN              NaN   

                           GDP  % Soldiers  Employment in industry  \
Country     Year                                                     
Afghanistan 2000   3342.034168    7.887961                    9.48   
            2001   3598.470576    5.020511                    8.98   
            2002   4141.523943    2.153062                    9.99   
            2003   4729.042179    2.208290                   10.35   
            2004   5388.482107    0.435599                   10.61   
...                        ...         ...                     ...   
Zimbabwe    2016  20548.678073    0.759750                    7.05   
            2017  22040.902301    0.750720                    6.90   
            2018  24311.560545    0.738210                    6.75   
            2019  21935.075306    0.738210                    6.57   
            2020  21786.743637    0.738210                    6.57   

                  Employment in services  Birth Rate  Cost business start-up  \
Country     Year                                                               
Afghanistan 2000               24.680000      48.021                    72.0   
            2001               24.719999      47.505                    72.0   
            2002               25.590000      46.901                    72.0   
            2003               25.950001      46.231                    72.0   
            2004               26.120001      45.507                    72.0   
...                                  ...         ...                     ...   
Zimbabwe    2016               26.070000      32.864                   121.5   
            2017               26.629999      31.732                   110.0   
            2018               27.230000      30.676                   110.7   
            2019               27.240000      29.747                    76.6   
            2020               27.240000      28.977                    76.6   

                  Death Rate  ...  Researchers in R&D  R&D expenditure %GDP  \
Country     Year              ...                                             
Afghanistan 2000      11.718  ...                 NaN                   NaN   
            2001      11.387  ...                 NaN                   NaN   
            2002      11.048  ...                 NaN                   NaN   
            2003      10.704  ...                 NaN                   NaN   
            2004      10.356  ...                 NaN                   NaN   
...                      ...  ...                 ...                   ...   
Zimbabwe    2016       8.286  ...                 NaN                   NaN   
            2017       8.044  ...                 NaN                   NaN   
            2018       7.883  ...                 NaN                   NaN   
            2019       7.773  ...                 NaN                   NaN   
            2020       7.692  ...                 NaN                   NaN   

                  % Rural Population  Tertiary School Gender P

AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  \
Country     Region             Year                                  
Afghanistan South Asia         2000              NaN   3342.034168   
                               2001              NaN   3598.470576   
                               2002              NaN   4141.523943   
                               2003              NaN   4729.042179   
                               2004              NaN   5388.482107   
...                                              ...           ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073   
                               2017              NaN  22040.902301   
                               2018              NaN  24311.560545   
                               2019              NaN  21935.075306   
                               2020              NaN  21786.743637   

                                     % Soldiers  Employment in industry  \
Country     Region             Year                                       
Afghanistan South Asia         2000    7.887961                    9.48   
                               2001    5.020511                    8.98   
                               2002    2.153062                    9.99   
                               2003    2.208290                   10.35   
                               2004    0.435599                   10.61   
...                                         ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016    0.759750                    7.05   
                               2017    0.750720                    6.90   
                               2018    0.738210                    6.75   
                               2019    0.738210                    6.57   
                               2020    0.738210                    6.57   

                                     Employment in services  Birth Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000               24.680000      48.021   
                               2001               24.719999      47.505   
                               2002               25.590000      46.901   
                               2003               25.950001      46.231   
                               2004               26.120001      45.507   
...                                                     ...         ...   
Zimbabwe    Sub-Saharan Africa 2016               26.070000      32.864   
                               2017               26.629999      31.732   
                               2018               27.230000      30.676   
                               2019               27.240000      29.747   
                               2020               27.240000      28.977   

                                     Cost business start-up  Death Rate  ...  \
Country     Region             Year                                      ...   
Afghanistan 

## EXPORT THE SEGREGATED DATAFRAMES
We will export the new DataFrame with the regions and two new kinds of DataFrame: one for every different country, and another for every region.

In [6]:
df.to_csv(write_path + '/GoldDataframe.csv')

country_list = set(df.index.get_level_values(col_country))
region_list = set(df.index.get_level_values(col_region))

for country in country_list:
    df.loc[df.index.get_level_values(col_country) == country].to_csv(country_folder + country + '.csv')

for region in region_list:
    df.loc[df.index.get_level_values(col_region) == region].to_csv(region_folder + region + '.csv')

# REGION AGGREGATES
In order to create an aggregated DataFrame with entries for the regions we are studying, we will have to estimate the values of its indicators: the summatory for absolute ones, and a weighted average based on population for the relative ones. The weight of the mean could also be established as the GDP of each country, or any other figure we consider representative enough of the country's importance in the calculus.

In [7]:
#Define which indicators should be not a weighted average, but a summatory.
abs_indicators = ['GDP', 'Population', 'TotalAgri']

#Wether aggregate from the regions aggregate (faster) or from the country raw data (slower but more precise).
fast_aggregate = True

#Initialize list to store the aggregated Dataframe for each region to later concatenate them all.
aggregated_list = []

for region in region_list:
    #Create sub-DataFrame with only the countries for each region
    region_df = df.loc[df.index.get_level_values(col_region) == region]
    #Aggregate by the countries of said region
    aggregated_region_df = aggregate(region_df, aggregate_by = col_country, for_index = col_year, new_group_col_name = col_region, group_name = region, abs_indicators = abs_indicators)
    #Add the aggregated DataFrame to the DataFrame per aggregated regions
    aggregated_list.append(aggregated_region_df)

aggregate_df = pd.concat(aggregated_list).sort_index()

#Obtain an aggregate for the world
col_world = 'World'

if fast_aggregate:
    aggregate_df_world = aggregate(aggregate_df, col_region, col_year, new_group_col_name = col_region, group_name = col_world, abs_indicators = abs_indicators)
else:
    aggregate_df_world = aggregate(df, col_country, col_year, new_group_col_name = col_region, group_name = col_world, abs_indicators = abs_indicators)

#Export the aggregated DataFrame to a .csv file
aggregate_df.to_csv(write_path + 'AggregatedRegion_DataFrame.csv')

aggregate_df_world.to_csv(write_path + 'AggregatedWorld_DataFrame.csv')

## ANALYSE AGGREGATION TIME
Since it is a time consuming process, the total time needed can be measured in the following cell if desired. Otherwised, it will be ignored.

In [8]:
#Set True if you want to analyse the time for both methods. By default is False to save that time.
test_time = False

if test_time:
    #Count time for the fast method: aggregating from the aggregated regions.
    st_time = datetime.now()
    fast_aggregate_df = aggregate(aggregate_df, col_region, col_year, new_group_col_name = col_region, group_name = 'World', abs_indicators = abs_indicators)
    f_time = datetime.now()
    fast_time = f_time - st_time
    print('Fast aggregation time: ' + fast_time.total_seconds())

    #Count time for the most precise method: aggregating from the raw data per country.
    st_time = datetime.now()
    precise_aggregate_df = aggregate(df, col_country, col_year, new_group_col_name = col_region, group_name = 'World',abs_indicators = abs_indicators)
    f_time = datetime.now()
    precise_time = f_time - st_time
    print('Precisse aggregation time: ' + precise_time.total_seconds())

    #Print the results to compare.
    display(fast_aggregate_df)
    display(precise_aggregate_df)

    #Export the results to .csv files if a more detailed analysis is needed.
    fast_aggregate_df.to_csv(region_folder + 'World_Fast.csv')
    precise_aggregate_df.to_csv(region_folder + 'World.csv')
else:
    print('No aggregation time analysed. If you want to study it, set the test_time variable to True and execute again')

No aggregation time analysed. If you want to study it, set the test_time variable to True and execute again


### Variables that can be changed
The p-value can be changed to a desired value. For the default value is recommnended 0.05. If you desire a higher confidence level lower this variable. This variable reffers directly to significance level, but for porpouses of clarification is set to this name:

We do the %store in order to pass this variable to other notebooks.

In [9]:
PVALUE_VAR = 0.05

%store PVALUE_VAR

Stored 'PVALUE_VAR' (float)


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. 
This codeblock also calculates the p-value of Spearman and Pearson, if the value > PVALUE_VAR the correlation will be deleted due to not having statiscal significance.

Later on is concatenated and generates the following result:

## Saving Correlation Dataframe Pearson

Use the same code as before, but this time save the correlation dataframe.

In [10]:
df = pd.read_csv(write_path + 'GoldDataframe.csv', index_col = [col_country, col_region, col_year])
#List of Series that will be concatenated.
corr_list = []

for country in country_list:
    #Get the DataFrame for a given country.
    country_df = df.loc[df.index.get_level_values(col_country) == country]

    #Correlation matrix for that country, using Pearson.
    country_corr_df = country_df.corr(method = 'pearson')

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p


    #Rename to country and remove unnecessary entries.
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_gdp])

    #Trim it into a single row (pd.Series) and append it to the list.
    country_corr_series = country_corr_df.loc[:, country]
    corr_list.append(country_corr_series)

#Concat all the series, transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = pd.concat(corr_list, axis = 1).transpose().sort_index()
corr_df.index.names = [col_country]
corr_df.to_csv(output_path + '/Corr_DF_pearson.csv')
corr_df

KeyError: 'Requested level (Country) does not match index name (None)'

Saving the correlation dataframe using spearman.

In [ ]:
df = pd.read_csv(write_path + 'GoldDataframe.csv')
#List of Series that will be concatenated.
corr_list = []

for country in country_list:
    #Get the DataFrame for a given country.
    country_df = df.loc[df.index.get_level_values(col_country) == country]

    #Correlation matrix for that country, using Spearman.
    country_corr_df = country_df.corr(method = 'spearman')

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: spearmanr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Rename to country and remove unnecessary entries.
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_gdp])

    #Trim it into a single row (pd.Series) and append it to the list.
    country_corr_series = country_corr_df.loc[:, country]
    corr_list.append(country_corr_series)

#Concat all the series, transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = pd.concat(corr_list, axis = 1).transpose().sort_index()
corr_df.index.names = [col_country]
corr_df.to_csv(output_path + '/Corr_DF_spearman.csv')
corr_df

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,-0.864566,NaN,NaN,NaN,NaN,-0.914286,-0.914286,NaN,0.914286,-0.905542,...,NaN,-0.914286,NaN,0.666451,NaN,-0.725737,NaN,NaN,NaN,0.914286
Albania,-0.486522,NaN,NaN,0.919481,-0.782836,-0.854823,0.885714,-0.577374,0.889610,-0.782415,...,NaN,-0.889610,0.796368,NaN,NaN,0.674271,NaN,NaN,NaN,-0.888312
Algeria,NaN,NaN,NaN,0.916883,-0.755527,0.874026,NaN,NaN,0.750649,NaN,...,0.797403,-0.750649,0.696313,NaN,0.629927,0.756356,NaN,NaN,NaN,0.750649
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.879064
Angola,NaN,NaN,NaN,NaN,-0.755527,-0.753247,-0.753247,NaN,0.753247,-0.776551,...,NaN,-0.753247,0.744056,0.748459,NaN,NaN,NaN,NaN,NaN,0.753247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uzbekistan,-0.714518,NaN,NaN,NaN,-0.672302,0.607537,-0.675026,NaN,0.850649,-0.812382,...,0.759740,-0.836364,NaN,NaN,0.925625,0.904891,NaN,NaN,NaN,0.850649
Vanuatu,NaN,-0.569106,NaN,0.798701,NaN,-0.971429,-0.944120,NaN,0.971429,-0.975901,...,NaN,-0.971429,0.827089,NaN,-0.813901,0.471664,NaN,NaN,NaN,0.971429
Yemen,NaN,NaN,NaN,0.657143,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.757143,0.836364,NaN,NaN,NaN,0.546753


## Cleaned GoldDataframe 
Before the correlation dataframe was generated and it detected all the correlations with a high p-value. To perform all the following analysis a clean GoldDataframe is needed. It simply detects if the correlation for the indicator and country is Nan and procedes to delete the whole column indicator for the country.

We use the spearman method.

In [ ]:
for country in country_list:
    for ind in corr_df.columns:
        if np.isnan(corr_df.loc[country, ind]):
            df.loc[df.index.get_level_values(col_country) == country, [ind]] = np.NaN

df.to_csv(write_path + 'GoldDataframe_significant.csv')